In [1]:
from datasets import load_dataset, Value, Sequence, Features

data_files={"train": r"C:\Users\jimmc\Documents\UCSD_Bootcamp\Hugging_Face\sample_data\train.json",
           "validation": r"C:\Users\jimmc\Documents\UCSD_Bootcamp\Hugging_Face\sample_data\val.json",
           "test": r"C:\Users\jimmc\Documents\UCSD_Bootcamp\Hugging_Face\sample_data\test.json"}
        
#data_files={"train": r"C:\Users\jimmc\Documents\UCSD_Bootcamp\Hugging_Face\sample_data\train.json"}


#dataset = load_dataset("json", data_files=data_files)

In [2]:
features = Features({'answers': Sequence(feature={'text': Value(dtype='string'), 'answer_start': Value(dtype='int32')}),
'context': Value(dtype='string'),
'id': Value(dtype='string'),
'question': Value(dtype='string'),
'title': Value(dtype='string')})

In [12]:
dataset = load_dataset("json", data_files=data_files)

Using custom data configuration default-0eba7e56df251bfe
Found cached dataset json (C:/Users/jimmc/.cache/huggingface/datasets/json/default-0eba7e56df251bfe/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
answers = dataset['train']['answers'][0]

In [5]:
answers

[{'answer_start': 253, 'text': 'The Unicode Standard'}]

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [7]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",     #only truncates the context nothing else
        return_offsets_mapping=True,  # map the start and end positions of the answer to the original context 
        padding="max_length",
    )
    
    offset_mapping = inputs.pop("offset_mapping")
    #print(offset_mapping)
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    #print(answers)
    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        #start_char = answer["answer_start"][0]
        #end_char = answer["answer_start"][0] + len(answer["text"][0])  #start char + length giver chars with answer
        start_char = answer[0]["answer_start"]
        end_char = answer[0]["answer_start"] + len(answer[0]["text"])  
        sequence_ids = inputs.sequence_ids(i)  #no clue what this does
        #print(answer, end_char, sequence_ids)
        # Find the start and end of the context        ; not sure how this section works
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1
    
        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)
            
            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)
            
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [32]:
tokenized_squad = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [33]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [34]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForQuestionAnswering: ['vocab_projector', 'vocab_transform', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 2
total_train_steps = (len(tokenized_squad["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

In [37]:
import tensorflow as tf

model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [38]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

In [39]:
tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["validation"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [40]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3)

Epoch 1/3
787/787 [==============================] - 19962s 25s/step - loss: 2.4348 - val_loss: 1.6549
Epoch 2/3
787/787 [==============================] - 19591s 25s/step - loss: 1.3575 - val_loss: 1.5875
Epoch 3/3
787/787 [==============================] - 19638s 25s/step - loss: 1.1994 - val_loss: 1.5875


In [121]:
model.summary()

Model: "tf_distil_bert_for_question_answering"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 qa_outputs (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66,364,418
Trainable params: 66,364,418
Non-trainable params: 0
_________________________________________________________________


In [16]:
#This does not work!!!!!!!!  See at the bottom for the correct way to save and reload models for use
#model.save(r"C:\Users\jimmc\Documents\UCSD_Bootcamp\Hugging_Face\hugging_face_testing",save_format="tf")

In [47]:
from transformers import pipeline
question_answerer = pipeline(task="question-answering", model=model, tokenizer=tokenizer)

In [57]:
dataset['test'][0]['answers']

[{'answer_start': 15, 'text': 'ten'}]

In [50]:
result = question_answerer(question=dataset['test'][0]['question'],context=dataset['test'][0]['context'])

In [51]:
result

{'score': 0.5929886102676392, 'start': 15, 'end': 18, 'answer': 'ten'}

In [78]:
exact_answers = []
contains_answer = []
wrong = []
scores = []
for text_row in dataset['test']:
    result = question_answerer(question = text_row['question'], context = text_row['context'])
    correct_start = text_row['answers'][0]['answer_start'] 
    correct_end = text_row['answers'][0]['answer_start'] + len(text_row['answers'][0]['text'])
    scores.append(result['score'])
   
    if (correct_start == result['start']) and (correct_end == result['end']):
        exact_answers.append(1)
        contains_answer.append(1)
        wrong.append(0)
   
    
    elif (correct_start != result['start']) or (correct_end != result['end']):
        
        if (correct_start >= result['start']) and (correct_end <= result['end']):
            exact_answers.append(0)
            contains_answer.append(1)
            wrong.append(0)
        else:
            exact_answers.append(0)
            contains_answer.append(0)
            wrong.append(1)
    else:
        pass

In [99]:
Accuracy = sum(exact_answers)/len(dataset['test'])
Accuracy

0.4568726937269373

In [80]:
sum(contains_answer)/len(dataset['test'])

0.5761070110701108

In [86]:
import numpy as np

In [89]:
np_scores = np.array(scores)
np_exact = np.array(exact_answers)
np_contains = np.array(contains_answer)
np_wrong = np.array(wrong)

In [112]:
#np.sum(np_exact[np_scores >= .5])
#len(np_exact[np_scores >= .5])
TPE = np.sum(np_exact[np_scores >= .5])
TPE
FPE = len(np_exact[np_scores >= .5]) - TPE
FPE
Exact_Percison = TPE/(TPE+FPE)
Exact_Percison

0.7634560906515581

In [115]:
TPC = np.sum(np_contains[np_scores >= .5])
TPC
FPC = len(np_contains[np_scores >= .5]) - TPC
FPC
Containing_Percison = TPC/(TPC+FPC)
Containing_Percison

0.8101983002832861

In [113]:
FNE = np.sum(np_exact[np_scores < .5])
FNE
TNE = len(np_exact[np_scores < .5]) - FNE
TNE
Exact_Recall = TPE/(TPE + FNE)
Exact_Recall

0.5441696113074205

In [116]:
FNC = np.sum(np_exact[np_scores < .5])
FNC
TNC = len(np_exact[np_scores < .5]) - FNC
TNC
Containing_Recall = TPC/(TPC + FNC)
Containing_Recall

0.558866634098681

In [127]:
#https://huggingface.co/docs/transformers/v4.23.1/en/serialization
model.save_pretrained(r"C:\Users\jimmc\Documents\UCSD_Bootcamp\Hugging_Face")
tokenizer.save_pretrained(r"C:\Users\jimmc\Documents\UCSD_Bootcamp\Hugging_Face")

# Type this below in the command line
# python -m transformers.onnx --model=C:\Users\jimmc\Documents\UCSD_Bootcamp\Hugging_Face onnx/

('C:\\Users\\jimmc\\Documents\\UCSD_Bootcamp\\Hugging_Face\\tokenizer_config.json',
 'C:\\Users\\jimmc\\Documents\\UCSD_Bootcamp\\Hugging_Face\\special_tokens_map.json',
 'C:\\Users\\jimmc\\Documents\\UCSD_Bootcamp\\Hugging_Face\\vocab.txt',
 'C:\\Users\\jimmc\\Documents\\UCSD_Bootcamp\\Hugging_Face\\added_tokens.json',
 'C:\\Users\\jimmc\\Documents\\UCSD_Bootcamp\\Hugging_Face\\tokenizer.json')

In [15]:
#You can Reload and use the model like below

In [9]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained(r'C:\Users\jimmc\Documents\UCSD_Bootcamp\Hugging_Face')

All model checkpoint layers were used when initializing TFDistilBertForQuestionAnswering.

All the layers of TFDistilBertForQuestionAnswering were initialized from the model checkpoint at C:\Users\jimmc\Documents\UCSD_Bootcamp\Hugging_Face.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(r'C:\Users\jimmc\Documents\UCSD_Bootcamp\Hugging_Face')

In [11]:
from transformers import pipeline
question_answerer = pipeline(task="question-answering", model=model, tokenizer=tokenizer)

In [13]:
result = question_answerer(question=dataset['test'][0]['question'],context=dataset['test'][0]['context'])

In [14]:
result

{'score': 0.5929886102676392, 'start': 15, 'end': 18, 'answer': 'ten'}